In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('seaborn')
import geopandas as gpd
%matplotlib inline 
pd.plotting.register_matplotlib_converters()
import seaborn as sns
print("Setup Complete")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Setup Complete


Import the data (make sure you update this on a daily basis)

In [2]:
conf_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
reco_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [43]:
conf_dfFL = conf_df.drop(['Province/State'],axis =1)
conf_dfFL = conf_dfFL.groupby('Country/Region').sum().reset_index()
conf_dfFL['Country/Region'] = conf_dfFL['Country/Region'].replace('Korea, South', 'South Korea')
conf_dfFL['Country/Region'] = conf_dfFL['Country/Region'].replace('Mainland China', 'China')
conf_dfFL.to_csv('flourishcovid_19_confirmedFL.csv', index=False)

In [44]:
death_dfFL = death_df.drop(['Province/State'],axis =1)
death_dfFL = death_dfFL.groupby('Country/Region').sum().reset_index()
death_dfFL['Country/Region'] = death_dfFL['Country/Region'].replace('Korea, South', 'South Korea')
death_dfFL['Country/Region'] = death_dfFL['Country/Region'].replace('Mainland China', 'China')
death_dfFL.to_csv('flouurishcovid_19_deathsFL.csv', index=False)

In [45]:
reco_dfFL = reco_df.drop(['Province/State'],axis =1)
reco_dfFL = reco_dfFL.groupby('Country/Region').sum().reset_index()
reco_dfFL['Country/Region'] = conf_dfFL['Country/Region'].replace('Korea, South', 'South Korea')
reco_dfFL['Country/Region'] = conf_dfFL['Country/Region'].replace('Mainland China', 'China')
reco_dfFL.to_csv('flourishcovid_19_recovred.csv', index=False)

In [46]:
dates = conf_df.columns[4:]

conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = death_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = reco_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
                       axis=1, sort=False)

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0


In [47]:
full_table = full_table[full_table['Province/State'].str.contains(',')!=True]
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')
full_table.to_csv('covid_19_full_updated.csv', index=False)

In [48]:
full_table = pd.read_csv('covid_19_full_updated.csv')

In [49]:
full_table.head(5)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0
